# importing important libraries

### Steps to be followed
    1.importing necessary libraries
    2.Crearing the S3 bucket
    3.Mapping train and Test data in S3
    4.Mapping the path of the models in S3

In [2]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri 
from sagemaker.session import s3_input, Session

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:172: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[05/06/25 23:04:46] INFO     Found credentials from IAM Role:                                   ]8;id=882464;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=865745;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [13]:
import time
# bucket_name = 'bankapplication1' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
bucket_name = f"my-unique-bucket-omkar-{int(time.time())}"

my_region = boto3.session.Session().region_name # set the region of the instance
print(my_region)

us-east-2


In [14]:
import boto3

s3 = boto3.client('s3', region_name='us-east-2')  # use client instead of resource for bucket creation

try:
    if my_region == 'us-east-2':
        s3.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={'LocationConstraint': 'us-east-2'}
        )
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ', e)


S3 bucket created successfully


In [15]:

# set an output path where the trained model will be saved
prefix = 'xgboost-as-a-built-in-algo'
output_path ='s3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://my-unique-bucket-omkar-1746573322/xgboost-as-a-built-in-algo/output


### Downloading The Dataset And Storing in S3

In [16]:

import pandas as pd
import urllib
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [17]:
### Train Test split

import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)
     

(28831, 61) (12357, 61)


In [19]:
### Saving Train And Test Into Buckets
## We start with Train Data
from sagemaker.inputs import TrainingInput
import os
import boto3
import pandas as pd

# Save train.csv
pd.concat(
    [train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)],
    axis=1
).to_csv('train.csv', index=False, header=False)

# Upload to S3
boto3.Session().resource('s3').Bucket(bucket_name).Object(
    os.path.join(prefix, 'train/train.csv')
).upload_file('train.csv')

# Use TrainingInput instead of s3_input
s3_input_train = TrainingInput(
    s3_data=f's3://{bucket_name}/{prefix}/train',
    content_type='csv'
)


[05/06/25 23:42:28] INFO     Found credentials from IAM Role:                                   ]8;id=951851;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=170589;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [20]:

# Test Data Into Buckets
from sagemaker.inputs import TrainingInput
import os
import boto3

# Save test.csv
pd.concat(
    [test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)],
    axis=1
).to_csv('test.csv', index=False, header=False)

# Upload to S3
boto3.Session().resource('s3').Bucket(bucket_name).Object(
    os.path.join(prefix, 'test/test.csv')
).upload_file('test.csv')

# Define S3 input using TrainingInput
s3_input_test = TrainingInput(
    s3_data=f's3://{bucket_name}/{prefix}/test',
    content_type='csv'
)


[05/06/25 23:44:10] INFO     Found credentials from IAM Role:                                   ]8;id=798714;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=364824;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

### Building Models Xgboot- Inbuilt Algorithm

In [27]:
# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
from sagemaker import image_uris

container = image_uris.retrieve(
    framework='xgboost',
    region=boto3.Session().region_name,
    version='1.0-1'
)


[05/07/25 01:00:59] INFO     Defaulting to only available Python version: py3                     ]8;id=829315;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=252319;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#610\610]8;;\

                    INFO     Defaulting to only supported image scope: cpu.                       ]8;id=763369;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=9311;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#534\534]8;;\

In [28]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [30]:

# construct a SageMaker estimator that calls the xgboost-container

from sagemaker.estimator import Estimator
from sagemaker import get_execution_role

estimator = Estimator(
    image_uri=container,  # not image_name
    role=get_execution_role(),
    instance_count=1,
    instance_type='ml.m5.2xlarge',
    volume_size=5,  # in GB
    max_run=300,
    max_wait=600,
    use_spot_instances=True,
    output_path=output_path,
    hyperparameters=hyperparameters
)



In [31]:

estimator.fit({'train': s3_input_train,'validation': s3_input_test})

[05/07/25 01:02:30] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=368015;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=63827;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name:                                       ]8;id=148747;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=720462;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\
                             sagemaker-xgboost-2025-05-07-01-02-30-369                                             

2025-05-07 01:02:31 Starting - Starting the training job...
2025-05-07 01:03:02 Downloading - Downloading input data...
2025-05-07 01:03:17 Downloading - Downloading the training image...
2025-05-07 01:03:58 Training - Training image download completed. Training in progress...[2025-05-07 01:04:15.434 ip-10-0-165-83.us-east-2.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Single node training

### Deploy Machine Learning Model As Endpoints


In [32]:

xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

[05/07/25 01:22:36] INFO     Creating model with name: sagemaker-xgboost-2025-05-07-01-22-36-865    ]8;id=702149;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=472239;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

[05/07/25 01:22:37] INFO     Creating endpoint-config with name                                     ]8;id=673583;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=348249;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#6019\6019]8;;\
                             sagemaker-xgboost-2025-05-07-01-22-36-865                                             

                    INFO     Creating endpoint with name sagemaker-xgboost-2025-05-07-01-22-36-865  ]8;id=337171;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=295234;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4841\4841]8;;\

---------!

#### Prediction of the Test Data

In [34]:
from sagemaker.serializers import CSVSerializer
import numpy as np

# Prepare test data
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values

# Configure the predictor
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = CSVSerializer()

# Perform prediction
predictions = xgb_predictor.predict(test_data_array).decode('utf-8')

# Convert predictions to numpy array
predictions_array = np.fromstring(predictions, sep=',')

print(predictions_array.shape)


(12357,)


In [35]:


predictions_array
     

array([0.05214286, 0.05660191, 0.05096195, ..., 0.03436061, 0.02942475,
       0.03715819])

In [36]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.7%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10785)    34% (151)
Purchase        9% (1124)     66% (297) 



### Deleting The Endpoints

In [37]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

[05/07/25 01:33:26] WARNING  The endpoint attribute has been renamed in sagemaker>=2.            ]8;id=140570;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=506694;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

                    INFO     Deleting endpoint with name: sagemaker-xgboost-2025-05-07-01-22-36-865 ]8;id=964627;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=798611;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4985\4985]8;;\

[{'ResponseMetadata': {'RequestId': 'SPTP435NCJEQ7DVS',
   'HostId': 'D2QZjBC8JY1n8afo4dABMsiNVWNfw30qX52PcavZzMkzV6RTX02/Vt/lmo7ElHWAomY7r2Cu2JV7NslWqPTkQA==',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'D2QZjBC8JY1n8afo4dABMsiNVWNfw30qX52PcavZzMkzV6RTX02/Vt/lmo7ElHWAomY7r2Cu2JV7NslWqPTkQA==',
    'x-amz-request-id': 'SPTP435NCJEQ7DVS',
    'date': 'Wed, 07 May 2025 01:33:27 GMT',
    'connection': 'close',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2025-05-07-01-02-30-369/debug-output/events/000000000000/000000000000_worker_0.tfevents'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2025-05-07-01-02-30-369/debug-output/index/000000000/000000000030_worker_0.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2025-05-07-01-02-30-369/debug-output/index/000000000/000000000020_wor